In [1]:
import sys
import os
from scipy.io import wavfile
import numpy as np
data_folder = "/Volumes/CB_RESEARCH/vocal_synthesis/"
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.init import *
from lasagne.nonlinearities import *
import cPickle as pickle
import experiment

//anaconda/lib/python2.7/site-packages/Theano-0.8.0.dev0-py2.7.egg/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
with open("out.model") as f:
    model = pickle.load(f)

In [3]:
args = dict()
args["batch_size"] = 128
args["chunk_size"] = 200
args["learning_rate"] = 0.01
args["momentum"] = 0.9
args["num_epochs"] = 20

symbols = experiment.prepare(args)
l_out = symbols["l_out"]

Number of params in model: 20201


In [4]:
lasagne.layers.set_all_param_values(l_out, model["params"])

In [5]:
fs, data = wavfile.read(data_folder + os.path.sep + "data.wav")
#data = np.asarray(data, dtype="float32")
mean = np.mean(data)
std = np.std(data)
data = (data - mean) / std
data = data.tolist()

In [6]:
get_out = symbols["get_out"]

In [13]:
seed_vector = data[5000:5000+200]
#get_out( np.asarray([data[0:200]], dtype="float32") )
print np.min(seed_vector)
print np.max(seed_vector)

-0.329727245721
0.341807784139


In [14]:
for b in range(0, 500000):
    next_value = get_out( np.asarray([ seed_vector[b::] ], dtype="float32") )[0][0]
    #print next_value
    seed_vector.append(next_value)

In [15]:
print np.min(seed_vector)
print np.max(seed_vector)

-0.329727245721
0.341807784139


In [17]:
len(seed_vector) / fs

31

In [18]:
?wavfile.write

In [27]:
result = (np.asarray(seed_vector)*std) + mean
result = np.asarray(result, dtype="int32")

In [28]:
result

array([419, 389, 336, ...,  28,  43,  59], dtype=int32)

In [34]:
wavfile.write(filename="/tmp/file.wav", rate=fs, data=result)

In [36]:
print np.min(result)
print np.max(result)

-707
733


In [37]:
print np.min(tmp)
print np.max(tmp)

-8936
9124


In [39]:
(505.0 - (-8936)) / (9124 - -8936)

0.5227574750830565